# Exploratory Data Analysis


## Dataloader exploration

In [1]:
import recsys.movie_lens

In [2]:
sequences, num_users, num_items, user_encoder, item_encoder = recsys.movie_lens.load_movielens_sequential(
    version="100k", data_dir=None, min_rating=4.0, min_interactions=5
)
sequences, num_users, num_items, user_encoder, item_encoder

Loaded 938 user sequences.
Users: 938, Items: 1448


({257: [253,
   284,
   296,
   183,
   171,
   759,
   106,
   902,
   115,
   198,
   174,
   208,
   354,
   314,
   179,
   98,
   12,
   97,
   152,
   166,
   177,
   178,
   467,
   1101,
   311,
   735,
   291,
   170,
   737,
   145,
   476,
   932,
   39],
  845: [293,
   536,
   465,
   288,
   145,
   119,
   729,
   402,
   807,
   898,
   857,
   289,
   107,
   121,
   127,
   248,
   296,
   9,
   246,
   407,
   747,
   808,
   238,
   360,
   607,
   587,
   809,
   1201,
   123,
   672,
   813,
   232,
   158,
   431,
   544,
   95,
   236,
   22,
   126,
   789,
   130,
   12,
   180,
   472,
   11,
   190,
   48,
   56,
   4,
   579,
   170,
   71,
   202,
   221,
   155,
   178,
   64,
   467,
   79,
   191,
   226,
   8,
   172,
   680,
   174,
   475,
   23,
   27,
   92,
   142,
   229,
   31,
   17,
   305,
   311,
   256,
   337,
   738,
   284,
   269,
   853,
   343,
   297,
   300,
   336,
   352,
   260,
   1226,
   301,
   324,
   333,
   330,
   1071,
 

In [3]:
train_loader, val_loader, test_loader, num_users, num_items = recsys.movie_lens.get_sequential_dataloaders(
    version="100k",
    batch_size=128,
    max_sequence_length=50,
    num_negatives=100,
    val_split="leave_one_out",
    test_split="leave_one_out",
)
train_loader, val_loader, test_loader, num_users, num_items

Loaded 938 user sequences.
Users: 938, Items: 1448


(<torch.utils.data.dataloader.DataLoader at 0x15870bef0>,
 938,
 1448)

Positive and negative samples are based on index

In [4]:
train_loader.dataset[1]


{'user_id': tensor(257),
 'input_sequence': tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0, 253, 284]),
 'target_item': tensor(296),
 'negative_samples': tensor([1366, 1365, 1298, 1199,  818,  680,  589,  587, 1115,  868,  707,  596,
          393, 1044,   44, 1213,  993, 1155, 1030,  510,  528, 1187,  514, 1276,
         1307, 1301,  678,    4, 1188,  923, 1084, 1346, 1316, 1255, 1114, 1175,
          195,  673,  231,  719,  755,  747,   38, 1422,   74, 1027,    9,  537,
         1305, 1092, 1165, 1438, 1384, 1051, 1258,   64, 1015,  124,  696,    6,
          905, 1386,  236, 1214,  831, 1245,  446, 1083,  431, 1380,  592,  896,
          839,  396, 1024,  242,  388, 1265,  344,  306,  501,  772, 1441,  474,
          175,  664,  362,  890,  233,  256,  222,  5

Dataloader outputs

In [5]:
batch = next(iter(train_loader))
batch

{'user_id': tensor([290, 222, 534, 403, 252, 930, 900, 580, 133, 143, 376, 388, 293, 187,
         753, 182, 434, 893, 589, 842, 442,  83, 484, 885, 199, 370, 926, 832,
         319,  12, 244,  21, 763, 840, 842, 278, 134, 217, 232, 280,  62,  48,
         309, 158,  10, 479, 827, 317,  22, 315, 413, 312, 434, 447, 300, 934,
         274, 493,  58, 286, 237, 520, 617, 779, 926, 639, 644, 485, 662, 736,
          23, 847, 318, 857, 580,  12, 733, 777, 300, 432, 293, 256, 858,   5,
         655, 247, 875,  59, 863, 370,  86, 386, 454, 888, 238, 448, 365,  89,
         753, 113, 258, 343, 923,  73, 447, 371, 308, 182, 375, 282, 681, 817,
         573, 894, 251, 343, 342, 678, 483, 232, 874, 209, 247, 433, 600, 566,
         930, 645]),
 'input_sequence': tensor([[  24, 1108,  826,  ...,  263,  151,  476],
         [   0,    0,    0,  ...,  718,  384,  707],
         [   0,    0,    0,  ...,  256,  826,  100],
         ...,
         [   0,    0,    0,  ...,    0,    0,  300],
         [   

## Models

In [6]:
from recsys.models import get_model
import yaml

model_config = """
architecture: "SASRec"
embedding_dim: 64
num_blocks: 2
num_heads: 2
dropout: 0.2
"""

model_config = yaml.safe_load(model_config)
model_config["num_users"] = num_users
model_config["num_items"] = num_items
model_config["max_sequence_length"] = 50

model = get_model(**model_config)

y_est = model(batch)
y_est.shape # (batch_size, num_negatives + 1)

torch.Size([128, 101])

In [7]:
import torch
import torch.nn as nn
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(10, 32, 512)
out = encoder_layer(src)
out.shape

torch.Size([10, 32, 512])

In [8]:
y_est

tensor([[ -5.8217,   3.4156,   4.8057,  ...,  -9.0532,   4.2960,  -1.1403],
        [-12.9526,  -3.8367,  -1.7260,  ...,  -1.3859,   4.8760,   3.2632],
        [-13.2487,   3.6315,  15.1768,  ...,  -3.8792,   5.4456,   0.9985],
        ...,
        [  5.2895, -15.0575,   1.7717,  ...,   5.7038,   9.5574, -10.4807],
        [  2.2313,  -6.0877,   4.5900,  ...,  -9.8328,   5.1396,  -8.8183],
        [ -3.2127,   0.4066,  -5.5221,  ...,  -5.6480, -11.9715,   5.3591]],
       grad_fn=<CatBackward0>)

## Training

In [ ]:
import recsys.train
import yaml

config = """
dataset:
  version: "100k"
  batch_size: 4096
  test_split: 0.2
  num_negatives: 4

model:
  # architecture: "NeuralColabFilter"
  # embedding_dim: 10
  # num_users: 943
  # num_items: 1682

  # architecture: "NeuralColabFilter"
  # mf_dim: 8
  # mlp_dim: 32
  # layers: [64, 32, 16]
  # include_bias: true

  architecture: "AutoInt"
  embedding_dim: 64
  num_heads: 8
  num_layers: 3
  dropout: 0.1
  include_bias: true
  categorical_features: null
  num_user_continuous: 0
  num_item_continuous: 0


training:
  epochs: 1
  learning_rate: 0.001
  loss_function: "mse"

logging:
  experiment_name: "movielens_rating_prediction"
  run_name: "debug"
"""
config = yaml.safe_load(config)
recsys.train.main(config)

INFO:recsys.train:Configuration: {'dataset': {'version': '100k', 'batch_size': 4096, 'test_split': 0.2, 'num_negatives': 4}, 'model': {'architecture': 'AutoInt', 'embedding_dim': 64, 'num_heads': 8, 'num_layers': 3, 'dropout': 0.1, 'include_bias': True, 'categorical_features': None, 'num_user_continuous': 0, 'num_item_continuous': 0}, 'training': {'epochs': 1, 'learning_rate': 0.001, 'loss_function': 'mse'}, 'logging': {'experiment_name': 'movielens_rating_prediction', 'run_name': 'debug'}}
INFO:recsys.train:Loading dataset: {'version': '100k', 'batch_size': 4096, 'test_split': 0.2, 'num_negatives': 4}
INFO:recsys.train:Loading model: AutoInt
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Loaded 55375 positive interactions
Users: 942, Items: 1447
Loaded user features with dims: {'continuous_dim': 2, 'categorical_dims': {'occupation': 21, 'zip_code': 795}}
Loaded item features with dims: {'continuous_dim': 19, 'categorical_dims': {'year': 71}}


/Users/rich/Developer/Github/VariousDataAnalysis/recommenders/recsys/.venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/rich/Developer/Github/VariousDataAnalysis/recommenders/recsys/notebooks/checkpoints exists and is not empty.

  | Name         | Type                   | Params | Mode 
----------------------------------------------------------------
0 | model        | Model                  | 203 K  | train
1 | criterion    | MSELoss                | 0      | train
2 | train_auc    | BinaryAUROC            | 0      | train
3 | val_auc      | BinaryAUROC            | 0      | train
4 | test_auc     | BinaryAUROC            | 0      | train
5 | train_pr_auc | BinaryAveragePrecision | 0      | train
6 | val_pr_auc   | BinaryAveragePrecision | 0      | train
7 | test_pr_auc  | BinaryAveragePrecision | 0      | train
----------------------------------------------------------------
203 K     Trainable params
0         Non-tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/rich/Developer/Github/VariousDataAnalysis/recommenders/recsys/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  1.91it/s]